<a href="https://colab.research.google.com/github/SebastianSaldarriagaC1/proyecto-final-modelos/blob/main/01_Limpieza_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Descargar Dataset

Dataset con el que vamos a trabajar:
https://www.kaggle.com/datasets/javagarm/fifa-19-complete-player-dataset

Inicialmente se debe tener la key de la API de Kaggle para descargar localmente el dataset. Para esto se puede seguir esta guía :

https://saturncloud.io/blog/how-to-use-kaggle-datasets-in-google-colab/

**Nota:** Solo es necesario seguirla hasta el paso 2, una vez descargado el Json con la Key se pueden empezar a ejecutar las celdas

In [ ]:
# Run this cell and select the kaggle.json file downloaded
# from the Kaggle account settings page.
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"sebassaldarriaga","key":"3bb04850fe5fccc1075f237932526fb6"}'}

In [ ]:
# Let's make sure the kaggle.json file is present.
!ls -lha kaggle.json

-rw-r--r-- 1 root root 72 Oct 11 20:49 kaggle.json


In [ ]:
# Next, install the Kaggle API client.
!pip install -q kaggle

In [ ]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d javagarm/fifa-19-complete-player-dataset

fifa-19-complete-player-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip fifa-19-complete-player-dataset.zip

Archive:  fifa-19-complete-player-dataset.zip
replace kl.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: kl.csv                  


In [ ]:
import pandas as pd
import sklearn as skl
import numpy as np

In [ ]:
df = pd.read_csv("kl.csv", encoding='latin-1')

# Entendimiento del Dataset

Inicialmente se va a hacer un análisis de las columnas para entender que es lo que contiene cada columna, el tipo de dato que debe tener cada columna y, por supuesto, identificar aquellas que nos interesan

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df.head()

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31.0,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94.0,94,FC Barcelona,https://cdn.sofifa.org/teams/2/light/241.png,110.5M,565K,2202,Left,5.0,4.0,4.0,Medium/ Medium,Messi,Yes,RF,10.0,"Jul 1, 2004",NaN,2021,5'7,159lbs,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,226.5M
1,1,20801,Cristiano Ronaldo,33.0,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94.0,94,Juventus,https://cdn.sofifa.org/teams/2/light/45.png,77M,405K,2228,Right,5.0,4.0,5.0,High/ Low,C. Ronaldo,Yes,ST,7.0,"Jul 10, 2018",NaN,2022,6'2,183lbs,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,127.1M
2,2,190871,Neymar Jr,26.0,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92.0,93,Paris Saint-Germain,https://cdn.sofifa.org/teams/2/light/73.png,118.5M,290K,2143,Right,5.0,5.0,5.0,High/ Medium,Neymar,Yes,LW,10.0,"Aug 3, 2017",NaN,2022,5'9,150lbs,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,228.1M
3,3,193080,De Gea,27.0,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91.0,93,Manchester United,https://cdn.sofifa.org/teams/2/light/11.png,72M,260K,1471,Right,4.0,3.0,1.0,Medium/ Medium,Lean,Yes,GK,1.0,"Jul 1, 2011",NaN,2020,6'4,168lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,138.6M
4,4,192985,K. De Bruyne,27.0,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91.0,92,Manchester City,https://cdn.sofifa.org/teams/2/light/10.png,102M,355K,2281,Right,4.0,5.0,4.0,High/ High,Normal,Yes,RCM,7.0,"Aug 30, 2015",NaN,2023,5'11,154lbs,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,196.4M


In [ ]:
df.describe(include = 'all')

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
count,18207.000000,18207.000000,18207,18206.000000,18207,18207,18207,18206.000000,18207.000000,17966,18207,18207,18207,18207.000000,18159,18159.000000,18159.000000,18159.000000,18159,18159,18159,18147,18147.000000,16654,1264,17918,18159,18159,16122,16122,16122,16122,16122,16122,16122,16122,16122,16122,16122,16122,16122,16122,16122,16122,16122,16122,16122,16122,16122,16122,16122,16122,16122,16122,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,18159.000000,16643
unique,NaN,NaN,17194,NaN,18207,164,164,NaN,NaN,651,679,217,144,NaN,2,NaN,NaN,NaN,9,10,2,27,NaN,1736,341,36,21,57,93,93,93,105,102,102,102,105,101,101,101,100,92,92,92,100,95,99,99,99,95,98,108,108,108,98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1244
top,NaN,NaN,J. Rodríguez,NaN,https://cdn.sofifa.org/players/4/19/158023.png,England,https://cdn.sofifa.org/flags/14.png,NaN,NaN,FC Barcelona,https://cdn.sofifa.org/teams/2/light/241.png,1.1M,1K,NaN,Right,NaN,NaN,NaN,Medium/ Medium,Normal,No,ST,NaN,"Jul 1, 2018",Atalanta,2019,6'0,165lbs,61+2,61+2,61+2,63+2,61+2,61+2,61+2,63+2,61+2,61+2,61+2,61+2,58+2,58+2,58+2,61+2,61+2,62+2,62+2,62+2,61+2,61+2,63+2,63+2,63+2,61+2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.1M
freq,NaN,NaN,11,NaN,1,1662,1662,NaN,NaN,33,33,431,4900,NaN,13948,NaN,NaN,NaN,9810,10595,16505,2152,NaN,1538,20,4819,2881,1483,705,705,705,745,713,713,713,745,744,744,744,804,762,762,762,804,713,660,660,660,713,669,634,634,634,669,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,557
mean,9103.000000,214298.338606,NaN,25.122048,NaN,NaN,NaN,66.237449,71.307299,NaN,NaN,NaN,NaN,1597.809908,NaN,1.113222,2.947299,2.361308,NaN,NaN,NaN,NaN,19.546096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.734181,45.550911,52.298144,58.686712,42.909026,55.371001,47.170824,42.863153,52.711933,58.369459,64.614076,64.726967,63.503607,61.836610,63.966573,55.460047,65.089432,63.219946,65.311967,47.109973,55.868991,46.698276,49.958478,53.400903,48.548598,58.648274,47.281623,47.697836,45.661435,16.616223,16.391596,16.232061,16.388898,16.710887,NaN
std,5256.052511,29965.244204,NaN,4.670022,NaN,NaN,NaN,6.907059,6.136496,NaN,NaN,NaN,NaN,272.586016,NaN,0.394031,0.660456,0.756164,NaN,NaN,NaN,NaN,15.947765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.364524,19.525820,17.379909,14.699495,17.694408,18.910371,18.395264,17.478763,15.327870,16.686595,14.927780,14.649953,14.766049,9.010464,14.136166,17.237958,11.820044,15.894741,12.557000,19.260524,17.367967,20.696909,19.529036,14.146

In [ ]:
df_columns = df.columns
df_columns

Index(['Unnamed: 0', 'ID', 'Name', 'Age', 'Photo', 'Nationality', 'Flag',
       'Overall', 'Potential', 'Club', 'Club Logo', 'Value', 'Wage', 'Special',
       'Preferred Foot', 'International Reputation', 'Weak Foot',
       'Skill Moves', 'Work Rate', 'Body Type', 'Real Face', 'Position',
       'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until',
       'Height', 'Weight', 'LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW',
       'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM',
       'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'Crossing',
       'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling',
       'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingT

Tal como se puede ver hay algunas columnas que contienen datos no importantes para nuestro modelo, tal como:


*   'Unnamed: 0': La cual es un conteo que traia el csv internamente
*   'ID': El ID de cada jugador dentro del dataset
*   'Name': Nombre de cada jugador
*   'Photo': Foto del jugador
*   'Flag': Bandera del país de cada jugador
*   'Club Logo': Logo del club al que pertenece cada jugador
*   'Work Rate':
*   'Body Type': Tipo de cuerpo de cada jugador
*   'Ral Face': Valor booleano que indica si el jugador tiene la cara real o un modelo 3D genérico
*   'Jersey Number': Número de camiseta del jugador en su equipo
*   'Joined': Fecha en que se unió a su club actual
*   'Loaned From': En case de que el jugador esté en préstamos, esta columna almacena el club que lo prestó






# Limpieza del Dataset

## Eliminar columnas que no nos interesan

In [ ]:
# Extraemos las columnas mencionadas anteriormente para eliminarlas del Dataset
non_important_columns = ['Unnamed: 0', 'ID', 'Name', 'Photo', 'Flag', 'Club Logo', 'Work Rate',
                         'Body Type', 'Real Face', 'Jersey Number', 'Joined', 'Loaned From']

In [ ]:
cleaned_df = df.drop(columns = non_important_columns)
cleaned_df

,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Position,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,31.0,Argentina,94.0,94,FC Barcelona,110.5M,565K,2202,Left,5.0,4.0,4.0,RF,2021,5'7,159lbs,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,226.5M
1,33.0,Portugal,94.0,94,Juventus,77M,405K,2228,Right,5.0,4.0,5.0,ST,2022,6'2,183lbs,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,127.1M
2,26.0,Brazil,92.0,93,Paris Saint-Germain,118.5M,290K,2143,Right,5.0,5.0,5.0,LW,2022,5'9,150lbs,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,228.1M
3,27.0,Spain,91.0,93,Manchester United,72M,260K,1471,Right,4.0,3.0,1.0,GK,2020,6'4,168lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,138.6M
4,27.0,Belgium,91.0,92,Manchester City,102M,355K,2281,Right,4.0,5.0,4.0,RCM,2023,5'11,154lbs,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,196.4M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,19.0,England,47.0,65,Crewe Alexandra,60K,1K,1307,Right,1.0,2.0,2.0,CM,2019,5'9,134lbs,42+2,42+2,42+2,44+2,44+2,44+2,44+2,44+2,45+2,45+2,45+2,44+2,45+2,45+2,45+2,44+2,44+2,45+2,45+2,45+2,44+2,45+2,45+2,45+2,45+2,45+2,34.0,38.0,40.0,49.0,25.0,42.0,30.0,34.0,45.0,43.0,54.0,57.0,60.0,49.0,76.0,43.0,55.0,40.0,47.0,38.0,46.0,46.0,39.0,52.0,43.0,45.0,40.0,48.0,47.0,10.0,13.0,7.0,8.0,9.0,143K
18203,19.0,Sweden,47.0,63,Trelleborgs FF,60K,1K,1098,Right,1.0,2.0,2.0,ST,2020,6'3,170lbs,45+2,45+2,45+2,39+2,42+2,42+2,42+2,39+2,40+2,40+2,40+2,38+2,35+2,35+2,35+2,38+2,30+2,31+2,31+2,31+2,30+2,29+2,32+2,32+2,32+2,29+2,23.0,52.0,52.0,43.0,36.0,39.0,32.0,20.0,25.0,40.0,41.0,39.0,38.0,40.0,52.0,41.0,47.0,43.0,67.0,42.0,47.0,16.0,46.0,33.0,43.0,42.0,22.0,15.0,19.0,10.0,9.0,9.0,5.0,12.0,113K
18204,16.0,England,47.0,67,Cambridge United,60K,1K,1189,Right,1.0,3.0,2.0,ST,2021,5'8,148lbs,45+2,45+2,45+2,45+2,46+2,46+2,46+2,45+2,44+2,44+2,44+2,44+2,38+2,38+2,38+2,44+2,34+2,30+2,30+2,30+2,34+2,33+2,28+2,28+2,28+2,33+2,25.0,40.0,46.0,38.0,38.0,45.0,38.0,27.0,28.0,44.0,70.0,69.0,50.0,47.0,

## Decisiones sobre los datos con NaN

In [ ]:
def columns_with_nulls_cleaned():
  null_counts = cleaned_df.isnull().sum()
  columns_with_nulls = null_counts[null_counts > 0]

  print(columns_with_nulls)

In [ ]:
columns_with_nulls_cleaned()

Age                            1
Overall                        1
Club                         241
Preferred Foot                48
International Reputation      48
                            ... 
GKHandling                    48
GKKicking                     48
GKPositioning                 48
GKReflexes                    48
Release Clause              1564
Length: 72, dtype: int64


## Jugadores específicos

Se nota extraño que en el dataset haya un jugador con edad desconocida y uno con media desconocida.

Veamos quienes son

In [ ]:
df[df['Overall'].isnull()]

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
18,18,192448,M. ter Stegen,26.0,https://cdn.sofifa.org/players/4/19/192448.png,Germany,https://cdn.sofifa.org/flags/21.png,NaN,92,FC Barcelona,https://cdn.sofifa.org/teams/2/light/241.png,58M,240K,1328,Right,3.0,4.0,1.0,Medium/ Medium,Normal,Yes,GK,22.0,"Jul 1, 2014",NaN,2022,6'2,187lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,14.0,11.0,36.0,14.0,17.0,18.0,12.0,42.0,18.0,38.0,50.0,37.0,85.0,43.0,22.0,79.0,35.0,79.0,10.0,43.0,22.0,11.0,69.0,25.0,69.0,25.0,13.0,10.0,87.0,85.0,88.0,85.0,90.0,123.3M


In [ ]:
cleaned_df[cleaned_df['Overall'].isnull()]

,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Position,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
18,26.0,Germany,NaN,92,FC Barcelona,58M,240K,1328,Right,3.0,4.0,1.0,GK,2022,6'2,187lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,14.0,11.0,36.0,14.0,17.0,18.0,12.0,42.0,18.0,38.0,50.0,37.0,85.0,43.0,22.0,79.0,35.0,79.0,10.0,43.0,22.0,11.0,69.0,25.0,69.0,25.0,13.0,10.0,87.0,85.0,88.0,85.0,90.0,123.3M


In [ ]:
df[df['Age'].isnull()]

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
11,11,182521,T. Kroos,NaN,https://cdn.sofifa.org/players/4/19/182521.png,Germany,https://cdn.sofifa.org/flags/21.png,90.0,90,Real Madrid,https://cdn.sofifa.org/teams/2/light/243.png,76.5M,355K,2190,Right,4.0,5.0,3.0,Medium/ Medium,Normal,Yes,LCM,8.0,"Jul 17, 2014",NaN,2022,6'0,168lbs,78+3,78+3,78+3,81+3,82+3,82+3,82+3,81+3,84+3,84+3,84+3,82+3,86+3,86+3,86+3,82+3,79+3,82+3,82+3,82+3,79+3,77+3,72+3,72+3,72+3,77+3,88.0,76.0,54.0,92.0,82.0,81.0,86.0,84.0,93.0,90.0,64.0,62.0,70.0,89.0,71.0,87.0,30.0,75.0,73.0,92.0,60.0,82.0,79.0,86.0,73.0,85.0,72.0,79.0,69.0,10.0,11.0,13.0,7.0,10.0,156.8M


In [ ]:
cleaned_df[cleaned_df['Age'].isnull()]

,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Position,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
11,NaN,Germany,90.0,90,Real Madrid,76.5M,355K,2190,Right,4.0,5.0,3.0,LCM,2022,6'0,168lbs,78+3,78+3,78+3,81+3,82+3,82+3,82+3,81+3,84+3,84+3,84+3,82+3,86+3,86+3,86+3,82+3,79+3,82+3,82+3,82+3,79+3,77+3,72+3,72+3,72+3,77+3,88.0,76.0,54.0,92.0,82.0,81.0,86.0,84.0,93.0,90.0,64.0,62.0,70.0,89.0,71.0,87.0,30.0,75.0,73.0,92.0,60.0,82.0,79.0,86.0,73.0,85.0,72.0,79.0,69.0,10.0,11.0,13.0,7.0,10.0,156.8M


Como vemos que son Toni Kroos y Marc-André Ter Stegen, dos jugadores muy específicos podemos llenar esos campos con los datos obtenidos de Internet sobre ellos en FIFA 19

M. Ter Stegen: https://sofifa.com/player/192448/marc-andre-ter-stegen/190075

T. Kroos: https://sofifa.com/player/192448/marc-andre-ter-stegen/190075


In [ ]:
# Ter Ttegen Overall 89
cleaned_df.loc[18, 'Overall'] = 89

In [ ]:
# Toni Kroos Edad 28
cleaned_df.loc[11, 'Age'] = 28

In [ ]:
columns_with_nulls_cleaned()

Club                         241
Preferred Foot                48
International Reputation      48
Weak Foot                     48
Skill Moves                   48
                            ... 
GKHandling                    48
GKKicking                     48
GKPositioning                 48
GKReflexes                    48
Release Clause              1564
Length: 70, dtype: int64


## Jugadores sin equipo

Al analizar las columnas que tienen valores nulos se hace extraño ver que hay jugadores sin equipos. Veamos cuales son


In [ ]:
cleaned_df[cleaned_df['Club'].isnull()]

,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Position,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
452,24.0,Argentina,80.0,85,NaN, 0, 0,2122,Right,2.0,4.0,4.0,CM,NaN,5'11,165lbs,71+2,71+2,71+2,75+2,75+2,75+2,75+2,75+2,77+2,77+2,77+2,76+2,79+2,79+2,79+2,76+2,75+2,77+2,77+2,77+2,75+2,74+2,72+2,72+2,72+2,74+2,76.0,55.0,60.0,84.0,73.0,78.0,79.0,78.0,82.0,82.0,75.0,69.0,77.0,74.0,77.0,82.0,61.0,79.0,69.0,80.0,79.0,72.0,74.0,82.0,57.0,74.0,73.0,75.0,72.0,9.0,14.0,6.0,9.0,10.0,NaN
538,33.0,Sweden,80.0,80,NaN, 0, 0,1797,Right,2.0,4.0,2.0,LCB,NaN,6'4,185lbs,62+2,62+2,62+2,56+2,58+2,58+2,58+2,56+2,58+2,58+2,58+2,57+2,64+2,64+2,64+2,57+2,68+2,74+2,74+2,74+2,68+2,70+2,79+2,79+2,79+2,70+2,49.0,51.0,81.0,73.0,37.0,49.0,36.0,40.0,67.0,63.0,46.0,49.0,55.0,76.0,36.0,74.0,64.0,67.0,83.0,59.0,81.0,82.0,54.0,49.0,79.0,78.0,82.0,83.0,79.0,7.0,9.0,12.0,10.0,15.0,NaN
568,26.0,Russia,79.0,81,NaN, 0, 0,1217,Right,1.0,3.0,1.0,GK,NaN,6'2,176lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,14.0,17.0,25.0,13.0,15.0,18.0,17.0,32.0,17.0,58.0,54.0,36.0,76.0,50.0,24.0,60.0,27.0,70.0,13.0,26.0,20.0,11.0,63.0,15.0,69.0,18.0,20.0,12.0,80.0,73.0,65.0,77.0,85.0,NaN
677,29.0,Russia,79.0,79,NaN, 0, 0,2038,Right,2.0,3.0,3.0,RB,NaN,5'10,154lbs,70+2,70+2,70+2,73+2,72+2,72+2,72+2,73+2,73+2,73+2,73+2,75+2,74+2,74+2,74+2,75+2,78+2,75+2,75+2,75+2,78+2,78+2,73+2,73+2,73+2,78+2,73.0,61.0,69.0,79.0,57.0,72.0,49.0,46.0,75.0,72.0,84.0,90.0,80.0,75.0,76.0,67.0,85.0,93.0,68.0,57.0,65.0,71.0,77.0,72.0,41.0,73.0,76.0,76.0,80.0,7.0,12.0,10.0,8.0,15.0,NaN
874,29.0,Russia,78.0,78,NaN, 0, 0,1810,Right,2.0,3.0,3.0,ST,NaN,6'5,201lbs,77+2,77+2,77+2,71+2,74+2,74+2,74+2,71+2,71+2,71+2,71+2,71+2,66+2,66+2,66+2,71+2,52+2,52+2,52+2,52+2,52+2,48+2,48+2,48+2,48+2,48+2,61.0,79.0,86.0,71.0,74.0,71.0,64.0,60.0,55.0,77.0,66.0,65.0,50.0,75.0,32.0,78.0,63.0,77.0,93.0,68.0,75.0,30.0,78.0,73.0,77.0,70.0,21.0,15.0,19.0,15.0,12.0,11.0,11.0,8.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17197,21.0,India,55.0,64,NaN, 0, 0,838,Right,1.0,2.0,1.0,GK,NaN,6'2,176lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,7.0,13.0,16.0,7.0,11.0,10.0,13.0,16.0,18.0,29.0,32.0,24.0,55.0,36.0,24.0,47.0,16.0,56.0,9.0,21.0,10.0,8.0,29.0,13.0,29.0,8.0,12.0,11.0,55.0,50.0,57.0,59.0,53.0,NaN
17215,26.0,Finland,55.0,57,NaN, 0, 0,1366,Right,1.0,3.0,2.0,RB,NaN,6'4,187lbs,46+2,46+2,46+2,45+2,44+2,44+2,44+2,45+2,43+2,43+2,43+2,46+2,43+2,43+2,43+2,46+2,51+2,49+2,49+2,49+2,51+2,53+2,57+2,57+2,57+2,53+2,42.0,41.0,65.0,45.0,26.0,50.0,30.0,32.0,35.0,40.0,65.0,55.0,58.0,49.0,42.0,29.0,61.0,56.0,76.0,26.0,48.0,54.0,46.0,36.0,33.0,36.0,51.0,58.0,64.0,11.0,9.0,9.0,12.0,12.0,NaN
17339,23.0,India,54.0,63,NaN, 0, 0,1321,Right,1.0,3.0,2.0,NaN,NaN,5'9,143lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.0,56.0,49.0,38.0,38.0,53.0,37.0,33.0,33.0,43.0,66.0,68.0,57.0,49.0,58.0,50.0,55.0,59.0,62.0,47.0,39.0,34.0,57.0,43.0,58.0,45.0,14.0,23.0,21.0,8.0,6.0,13.0,9.0,10.0,NaN
17436,20.0,India,54.0,67,NaN, 0, 0,1270,Right,1.0,3.0,2.0,NaN,NaN,6'0,168lbs,NaN,NaN,NaN,NaN,N

Como se puede ver estos jugadores son datos atípicos que presentan 0 en todas las columnas relacionadas con dinero. Esto los convierte en datos sumamente atípicos por lo que lo mejor será eliminarlos para evitar posibles errores de medición a futuro.

In [ ]:
cleaned_df = cleaned_df[cleaned_df['Club'].notnull()]

## Jugadores con muchas estadísticas desconocidas

In [ ]:
columns_with_nulls_cleaned()

Preferred Foot                48
International Reputation      48
Weak Foot                     48
Skill Moves                   48
Position                      48
                            ... 
GKHandling                    48
GKKicking                     48
GKPositioning                 48
GKReflexes                    48
Release Clause              1323
Length: 69, dtype: int64


Ahora, notamos que hay exactamente 48 jugadores con NaN en muchas de las columnas. Veamos cuales son...

In [ ]:
cleaned_df[cleaned_df['Preferred Foot'].isnull()]

,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Position,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
13236,33.0,Scotland,62.0,62,Rochdale,120K,1K,1510,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13237,29.0,Nicaragua,62.0,62,Boyacá Chicó FC,300K,1K,1532,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13238,35.0,England,62.0,62,Notts County,140K,3K,1573,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13239,20.0,Italy,62.0,72,Brescia,425K,1K,1610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13240,24.0,England,62.0,66,Hamilton Academical FC,400K,1K,1481,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13241,21.0,Poland,62.0,72,?l?sk Wroc?aw,425K,1K,1692,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13242,23.0,Argentina,62.0,70,Club Atlético Aldosivi,450K,2K,1663,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13243,19.0,England,62.0,78,Everton,600K,5K,1328,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13244,30.0,Denmark,62.0,62,Hobro IK,230K,2K,1244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13245,21.0,Brazil,62.0,69,HJK Helsinki,425K,1K,1549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Como se ve, estos son jugadores de los que apenas se sabe algo, por lo que lo mejor será eliminarlos del dataset

In [ ]:
cleaned_df = cleaned_df[cleaned_df['Crossing'].notnull()]

## Decisión sobre los porteros

In [ ]:
columns_with_nulls_cleaned()

LS                1992
ST                1992
RS                1992
LW                1992
LF                1992
CF                1992
RF                1992
RW                1992
LAM               1992
CAM               1992
RAM               1992
LM                1992
LCM               1992
CM                1992
RCM               1992
RM                1992
LWB               1992
LDM               1992
CDM               1992
RDM               1992
RWB               1992
LB                1992
LCB               1992
CB                1992
RCB               1992
RB                1992
Release Clause    1275
dtype: int64


Podemos ver que quedan 1992 jugadores con datos en NaN. Sin embargo estos corresponden a aquellos que juegan de porteros. Veamos...

In [ ]:
cleaned_df[cleaned_df['LS'].isnull()].get('Position').describe()

count     1992
unique       1
top         GK
freq      1992
Name: Position, dtype: object

In [ ]:
cleaned_df[cleaned_df['LS'].isnull()]

,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Position,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
3,27.0,Spain,91.0,93,Manchester United,72M,260K,1471,Right,4.0,3.0,1.0,GK,2020,6'4,168lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,138.6M
9,25.0,Slovenia,90.0,93,Atlético Madrid,68M,94K,1331,Right,3.0,3.0,1.0,GK,2021,6'2,192lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,11.0,15.0,29.0,13.0,12.0,13.0,14.0,26.0,16.0,43.0,60.0,67.0,86.0,49.0,22.0,76.0,41.0,78.0,12.0,34.0,19.0,11.0,70.0,11.0,70.0,27.0,12.0,18.0,86.0,92.0,78.0,88.0,89.0,144.5M
18,26.0,Germany,89.0,92,FC Barcelona,58M,240K,1328,Right,3.0,4.0,1.0,GK,2022,6'2,187lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,14.0,11.0,36.0,14.0,17.0,18.0,12.0,42.0,18.0,38.0,50.0,37.0,85.0,43.0,22.0,79.0,35.0,79.0,10.0,43.0,22.0,11.0,69.0,25.0,69.0,25.0,13.0,10.0,87.0,85.0,88.0,85.0,90.0,123.3M
19,26.0,Belgium,89.0,90,Real Madrid,53.5M,240K,1311,Left,4.0,2.0,1.0,GK,2024,6'6,212lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,14.0,13.0,33.0,12.0,13.0,19.0,20.0,35.0,23.0,46.0,52.0,61.0,84.0,45.0,36.0,68.0,38.0,70.0,17.0,23.0,15.0,13.0,44.0,27.0,66.0,20.0,18.0,16.0,85.0,91.0,72.0,86.0,88.0,113.7M
22,32.0,Germany,89.0,89,FC Bayern München,38M,130K,1473,Right,5.0,4.0,1.0,GK,2021,6'4,203lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,13.0,25.0,55.0,11.0,30.0,14.0,11.0,59.0,48.0,54.0,60.0,51.0,84.0,35.0,25.0,77.0,43.0,80.0,16.0,29.0,30.0,12.0,70.0,47.0,70.0,17.0,10.0,11.0,90.0,86.0,91.0,87.0,87.0,62.7M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18178,18.0,Sweden,48.0,65,Dalkurd FF,50K,1K,738,Right,1.0,2.0,1.0,GK,2018,6'0,176lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,7.0,11.0,22.0,6.0,8.0,10.0,8.0,20.0,12.0,24.0,28.0,33.0,40.0,29.0,15.0,48.0,20.0,38.0,6.0,20.0,10.0,7.0,18.0,12.0,28.0,16.0,11.0,10.0,47.0,46.0,50.0,45.0,51.0,94K
18180,22.0,Scotland,48.0,58,St. Johnstone FC,40K,1K,987,Right,1.0,2.0,1.0,GK,2019,6'1,172lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,15.0,15.0,26.0,14.0,13.0,12.0,19.0,19.0,25.0,40.0,42.0,45.0,47.0,52.0,25.0,55.0,33.0,58.0,20.0,20.0,29.0,13.0,20.0,29.0,28.0,12.0,15.0,16.0,45.0,49.0,50.0,50.0,45.0,78K
18183,44.0,England,48.0,48,Cambridge United, 0,1K,774,Right,1.0,2.0,1.0,GK,2023,6'2,190lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,12.0,12.0,18.0,11.0,11.0,12.0,13.0,13.0,16.0,15.0,17.0,31.0,48.0,31.0,23.0,45.0,22.0,44.0,12.0,24.0,15.0,10.0,11.0,22.0,56.0,15.0,15.0,13.0,45.0,48.0,44.0,49.0,46.0,NaN
18194,18.0,Italy,47.0,65,Lecce,50K,1K,731,Right,1.0,3.0,1.0,GK,2021,6'3,187lbs,NaN,N

In [ ]:
cleaned_df = cleaned_df[cleaned_df['Position'] != 'GK']

Como eliminamos a los porteros ahora las columnas con estadísticas específicas para esta posición no son necesarias

In [ ]:
cleaned_df.columns

Index(['Age', 'Nationality', 'Overall', 'Potential', 'Club', 'Value', 'Wage',
       'Special', 'Preferred Foot', 'International Reputation', 'Weak Foot',
       'Skill Moves', 'Position', 'Contract Valid Until', 'Height', 'Weight',
       'LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM',
       'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB',
       'LCB', 'CB', 'RCB', 'RB', 'Crossing', 'Finishing', 'HeadingAccuracy',
       'ShortPassing', 'Volleys', 'Dribbling', 'Curve', 'FKAccuracy',
       'LongPassing', 'BallControl', 'Acceleration', 'SprintSpeed', 'Agility',
       'Reactions', 'Balance', 'ShotPower', 'Jumping', 'Stamina', 'Strength',
       'LongShots', 'Aggression', 'Interceptions', 'Positioning', 'Vision',
       'Penalties', 'Composure', 'Marking', 'StandingTackle', 'SlidingTackle',
       'GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes',
       'Release Clause'],
      dtype='object')

In [ ]:
gk_columns = ['GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes']

In [ ]:
cleaned_df = cleaned_df.drop(columns = gk_columns)
cleaned_df

,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Position,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,Release Clause
0,31.0,Argentina,94.0,94,FC Barcelona,110.5M,565K,2202,Left,5.0,4.0,4.0,RF,2021,5'7,159lbs,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,226.5M
1,33.0,Portugal,94.0,94,Juventus,77M,405K,2228,Right,5.0,4.0,5.0,ST,2022,6'2,183lbs,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,127.1M
2,26.0,Brazil,92.0,93,Paris Saint-Germain,118.5M,290K,2143,Right,5.0,5.0,5.0,LW,2022,5'9,150lbs,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,228.1M
4,27.0,Belgium,91.0,92,Manchester City,102M,355K,2281,Right,4.0,5.0,4.0,RCM,2023,5'11,154lbs,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,196.4M
5,27.0,Belgium,91.0,91,Chelsea,93M,340K,2142,Right,4.0,4.0,4.0,LF,2020,5'8,163lbs,83+3,83+3,83+3,89+3,88+3,88+3,88+3,89+3,89+3,89+3,89+3,89+3,82+3,82+3,82+3,89+3,66+3,63+3,63+3,63+3,66+3,60+3,49+3,49+3,49+3,60+3,81.0,84.0,61.0,89.0,80.0,95.0,83.0,79.0,83.0,94.0,94.0,88.0,95.0,90.0,94.0,82.0,56.0,83.0,66.0,80.0,54.0,41.0,87.0,89.0,86.0,91.0,34.0,27.0,22.0,172.1M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,19.0,England,47.0,65,Crewe Alexandra,60K,1K,1307,Right,1.0,2.0,2.0,CM,2019,5'9,134lbs,42+2,42+2,42+2,44+2,44+2,44+2,44+2,44+2,45+2,45+2,45+2,44+2,45+2,45+2,45+2,44+2,44+2,45+2,45+2,45+2,44+2,45+2,45+2,45+2,45+2,45+2,34.0,38.0,40.0,49.0,25.0,42.0,30.0,34.0,45.0,43.0,54.0,57.0,60.0,49.0,76.0,43.0,55.0,40.0,47.0,38.0,46.0,46.0,39.0,52.0,43.0,45.0,40.0,48.0,47.0,143K
18203,19.0,Sweden,47.0,63,Trelleborgs FF,60K,1K,1098,Right,1.0,2.0,2.0,ST,2020,6'3,170lbs,45+2,45+2,45+2,39+2,42+2,42+2,42+2,39+2,40+2,40+2,40+2,38+2,35+2,35+2,35+2,38+2,30+2,31+2,31+2,31+2,30+2,29+2,32+2,32+2,32+2,29+2,23.0,52.0,52.0,43.0,36.0,39.0,32.0,20.0,25.0,40.0,41.0,39.0,38.0,40.0,52.0,41.0,47.0,43.0,67.0,42.0,47.0,16.0,46.0,33.0,43.0,42.0,22.0,15.0,19.0,113K
18204,16.0,England,47.0,67,Cambridge United,60K,1K,1189,Right,1.0,3.0,2.0,ST,2021,5'8,148lbs,45+2,45+2,45+2,45+2,46+2,46+2,46+2,45+2,44+2,44+2,44+2,44+2,38+2,38+2,38+2,44+2,34+2,30+2,30+2,30+2,34+2,33+2,28+2,28+2,28+2,33+2,25.0,40.0,46.0,38.0,38.0,45.0,38.0,27.0,28.0,44.0,70.0,69.0,50.0,47.0,58.0,45.0,60.0,55.0,32.0,45.0,32.0,15.0,48.0,43.0,55.0,41.0,32.0,13.0,11.0,165K
18205,17.0,England,47.0,66,Tranmere Rovers,60K,1K,1228,Right,1.0,3.0,2.0,RW,2019,5'10,154lbs,47+2,47+2,47+2,47+2,46+2,46+2,46+2,47+2,45+2

## Jugadores con Release Clause en NaN

In [ ]:
columns_with_nulls_cleaned()

Release Clause    1183
dtype: int64


Como vemos, la única columna que aún tiene datos nulos es Release Clause. Esto sucede ya que se trata de jugadores que en su momento estaban de prestamo en sus clubes, veamos...

In [ ]:
cleaned_df[cleaned_df['Release Clause'].isnull()]

,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Position,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,Release Clause
28,26.0,Colombia,88.0,89,FC Bayern München,69.5M,315K,2171,Left,4.0,3.0,4.0,LAM,"Jun 30, 2019",5'11,172lbs,80+3,80+3,80+3,84+3,83+3,83+3,83+3,84+3,85+3,85+3,85+3,83+3,81+3,81+3,81+3,83+3,69+3,68+3,68+3,68+3,69+3,65+3,58+3,58+3,58+3,65+3,90.0,83.0,62.0,89.0,90.0,85.0,89.0,86.0,83.0,90.0,73.0,67.0,83.0,85.0,76.0,86.0,54.0,70.0,68.0,92.0,64.0,55.0,80.0,89.0,81.0,87.0,52.0,41.0,44.0,NaN
38,30.0,Argentina,88.0,88,Milan,57M,245K,1965,Right,4.0,4.0,3.0,LS,"Jun 30, 2019",6'1,196lbs,85+3,85+3,85+3,81+3,83+3,83+3,83+3,81+3,80+3,80+3,80+3,78+3,71+3,71+3,71+3,78+3,55+3,53+3,53+3,53+3,55+3,51+3,48+3,48+3,48+3,51+3,68.0,92.0,80.0,75.0,90.0,84.0,74.0,62.0,59.0,85.0,73.0,73.0,75.0,86.0,69.0,86.0,79.0,70.0,85.0,80.0,50.0,20.0,92.0,74.0,70.0,86.0,35.0,22.0,18.0,NaN
91,29.0,Brazil,85.0,85,Guangzhou Evergrande Taobao FC,37M,235K,2245,Right,3.0,3.0,3.0,LDM,"Jun 30, 2019",6'0,157lbs,81+3,81+3,81+3,79+3,81+3,81+3,81+3,79+3,81+3,81+3,81+3,79+3,82+3,82+3,82+3,79+3,78+3,81+3,81+3,81+3,78+3,78+3,80+3,80+3,80+3,78+3,64.0,81.0,85.0,84.0,75.0,82.0,66.0,72.0,80.0,84.0,73.0,73.0,69.0,84.0,66.0,83.0,90.0,92.0,81.0,80.0,81.0,81.0,84.0,78.0,71.0,80.0,75.0,81.0,73.0,NaN
166,24.0,Brazil,83.0,90,Guangzhou Evergrande Taobao FC,36.5M,18K,2142,Left,2.0,4.0,4.0,CAM,"Dec 31, 2018",6'3,176lbs,80+2,80+2,80+2,81+2,81+2,81+2,81+2,81+2,81+2,81+2,81+2,80+2,79+2,79+2,79+2,80+2,69+2,69+2,69+2,69+2,69+2,67+2,64+2,64+2,64+2,67+2,78.0,80.0,77.0,81.0,76.0,82.0,86.0,87.0,79.0,84.0,77.0,76.0,76.0,79.0,65.0,84.0,58.0,76.0,72.0,88.0,67.0,58.0,86.0,81.0,71.0,74.0,55.0,62.0,42.0,NaN
176,24.0,Croatia,83.0,89,Chelsea,35M,165K,2115,Right,3.0,4.0,4.0,LCM,"Jun 30, 2019",5'9,172lbs,71+3,71+3,71+3,79+3,78+3,78+3,78+3,79+3,81+3,81+3,81+3,80+3,82+3,82+3,82+3,80+3,76+3,78+3,78+3,78+3,76+3,74+3,69+3,69+3,69+3,74+3,73.0,62.0,42.0,87.0,76.0,88.0,78.0,68.0,82.0,89.0,84.0,74.0,81.0,70.0,85.0,79.0,63.0,82.0,65.0,75.0,77.0,78.0,69.0,88.0,59.0,72.0,69.0,75.0,63.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17978,21.0,England,51.0,57,Hamilton Academical FC,50K,3K,1277,Right,1.0,2.0,2.0,ST,"Jun 30, 2019",5'11,165lbs,49+2,49+2,49+2,49+2,49+2,49+2,49+2,49+2,47+2,47+2,47+2,47+2,40+2,40+2,40+2,47+2,34+2,31+2,31+2,31+2,34+2,33+2,30+2,30+2,30+2,33+2,34.0,53.0,44.0,38.0,40.0,50.0,39.0,26.0,33.0,48.0,64.0,74.0,56.0,49.0,65.0,42.0,62.0,48.0,60.0,43.0,35.0,16.0,47.0,45.0,62.0,49.0,14.0,13.0,13.0,NaN
17979,21.0,China PR,51.0,60,Guizhou Hengfeng FC,60K,2K,1459,Right,1.0,2.0,2.0,CM,"Dec 31, 2018",6'0,172lbs,49+2,49+2,49+2,50+2,50+2,50+2,50+2,50+2,50+2,50+2,50+2,51+2,50+2,50+2,50+2,51+2,50+2,50+2,50+2,50+2,50+2,50+2,49+2,49+2,49+2,50+2,36.0,30.0,48.0,57.0,32.0,45.0,37.0,38.0,55.0,50.0,71.0,76.0,75.0,55.0,58.0,55.0,60.0,62.0,61.0,33.0,58.0,41.0,53.0,47.0,36.0,43.0,47.0,41.0,46.0,NaN
18026,21.0,China PR,50.0,59,Guizhou Hengfeng FC,50K,2K,1440,Right,1.0,2.0,2.0,LM,"Dec 31, 2018",5'10,143lbs,44+2,44+2,44+2,47+2,46+2,46+2,46+2,47+2,48+2,48+2,48+2,49+2,48+2,48+2,48+2,49+2,50+2,50+2,50+2,50+2,50+2,51+2,50+2,50+2,50+2,51+2,41.0,28.0,50.0,54.0,26.0,50.0,39.0,40.0,55.0,46.0,61.0,77.0,76.0,50.0,74.0,47.0,63.0,50.0,52.0,29.0,56.0,50.0,37.0,49.0,42.0,38.0,46.0,48.0,51.0,NaN
18031,20.0,China PR,50.0,61,Stabæ

In [ ]:
df[~df['Loaned From'].isnull()]

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
28,28,198710,J. Rodríguez,26.0,https://cdn.sofifa.org/players/4/19/198710.png,Colombia,https://cdn.sofifa.org/flags/56.png,88.0,89,FC Bayern München,https://cdn.sofifa.org/teams/2/light/21.png,69.5M,315K,2171,Left,4.0,3.0,4.0,Medium/ Medium,Normal,Yes,LAM,10.0,NaN,Real Madrid,"Jun 30, 2019",5'11,172lbs,80+3,80+3,80+3,84+3,83+3,83+3,83+3,84+3,85+3,85+3,85+3,83+3,81+3,81+3,81+3,83+3,69+3,68+3,68+3,68+3,69+3,65+3,58+3,58+3,58+3,65+3,90.0,83.0,62.0,89.0,90.0,85.0,89.0,86.0,83.0,90.0,73.0,67.0,83.0,85.0,76.0,86.0,54.0,70.0,68.0,92.0,64.0,55.0,80.0,89.0,81.0,87.0,52.0,41.0,44.0,15.0,15.0,15.0,5.0,14.0,NaN
38,38,167664,G. Higuaín,30.0,https://cdn.sofifa.org/players/4/19/167664.png,Argentina,https://cdn.sofifa.org/flags/52.png,88.0,88,Milan,https://cdn.sofifa.org/teams/2/light/47.png,57M,245K,1965,Right,4.0,4.0,3.0,High/ Medium,Normal,Yes,LS,9.0,NaN,Juventus,"Jun 30, 2019",6'1,196lbs,85+3,85+3,85+3,81+3,83+3,83+3,83+3,81+3,80+3,80+3,80+3,78+3,71+3,71+3,71+3,78+3,55+3,53+3,53+3,53+3,55+3,51+3,48+3,48+3,48+3,51+3,68.0,92.0,80.0,75.0,90.0,84.0,74.0,62.0,59.0,85.0,73.0,73.0,75.0,86.0,69.0,86.0,79.0,70.0,85.0,80.0,50.0,20.0,92.0,74.0,70.0,86.0,35.0,22.0,18.0,5.0,12.0,7.0,5.0,10.0,NaN
91,91,187961,Paulinho,29.0,https://cdn.sofifa.org/players/4/19/187961.png,Brazil,https://cdn.sofifa.org/flags/54.png,85.0,85,Guangzhou Evergrande Taobao FC,https://cdn.sofifa.org/teams/2/light/111839.png,37M,235K,2245,Right,3.0,3.0,3.0,High/ High,Lean,Yes,LDM,9.0,NaN,FC Barcelona,"Jun 30, 2019",6'0,157lbs,81+3,81+3,81+3,79+3,81+3,81+3,81+3,79+3,81+3,81+3,81+3,79+3,82+3,82+3,82+3,79+3,78+3,81+3,81+3,81+3,78+3,78+3,80+3,80+3,80+3,78+3,64.0,81.0,85.0,84.0,75.0,82.0,66.0,72.0,80.0,84.0,73.0,73.0,69.0,84.0,66.0,83.0,90.0,92.0,81.0,80.0,81.0,81.0,84.0,78.0,71.0,80.0,75.0,81.0,73.0,16.0,16.0,11.0,6.0,8.0,NaN
166,166,212523,Anderson Talisca,24.0,https://cdn.sofifa.org/players/4/19/212523.png,Brazil,https://cdn.sofifa.org/flags/54.png,83.0,90,Guangzhou Evergrande Taobao FC,https://cdn.sofifa.org/teams/2/light/111839.png,36.5M,18K,2142,Left,2.0,4.0,4.0,High/ Medium,Normal,No,CAM,24.0,NaN,SL Benfica,"Dec 31, 2018",6'3,176lbs,80+2,80+2,80+2,81+2,81+2,81+2,81+2,81+2,81+2,81+2,81+2,80+2,79+2,79+2,79+2,80+2,69+2,69+2,69+2,69+2,69+2,67+2,64+2,64+2,64+2,67+2,78.0,80.0,77.0,81.0,76.0,82.0,86.0,87.0,79.0,84.0,77.0,76.0,76.0,79.0,65.0,84.0,58.0,76.0,72.0,88.0,67.0,58.0,86.0,81.0,71.0,74.0,55.0,62.0,42.0,13.0,11.0,13.0,12.0,10.0,NaN
176,176,207410,M. Kova?i?,24.0,https://cdn.sofifa.org/players/4/19/207410.png,Croatia,https://cdn.sofifa.org/flags/10.png,83.0,89,Chelsea,https://cdn.sofifa.org/teams/2/light/5.png,35M,165K,2115,Right,3.0,4.0,4.0,High/ High,Normal,Yes,LCM,17.0,NaN,Real Madrid,"Jun 30, 2019",5'9,172lbs,71+3,71+3,71+3,79+3,78+3,78+3,78+3,79+3,81+3,81+3,81+3,80+3,82+3,82+3,82+3,80+3,76+3,78+3,78+3,78+3,76+3,74+3,69+3,69+3,69+3,74+3,73.0,62.0,42.0,87.0,76.0,88.0,78.0,68.0,82.0,89.0,84.0,74.0,81.0,70.0,85.0,79.0,63.0,82.0,65.0,75.0,77.0,78.0,69.0,88.0,59.0,72.0,69.0,75.0,63.0,9.0,5.0,6.0,7.0,7.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

A estos jugadores vamos a asignarles una Release Clause basada en su valor de la siguiente forma...

In [ ]:
cleaned_df = cleaned_df[~cleaned_df['Release Clause'].isnull()]
cleaned_df

,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Position,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,Release Clause
0,31.0,Argentina,94.0,94,FC Barcelona,110.5M,565K,2202,Left,5.0,4.0,4.0,RF,2021,5'7,159lbs,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,226.5M
1,33.0,Portugal,94.0,94,Juventus,77M,405K,2228,Right,5.0,4.0,5.0,ST,2022,6'2,183lbs,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,127.1M
2,26.0,Brazil,92.0,93,Paris Saint-Germain,118.5M,290K,2143,Right,5.0,5.0,5.0,LW,2022,5'9,150lbs,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,228.1M
4,27.0,Belgium,91.0,92,Manchester City,102M,355K,2281,Right,4.0,5.0,4.0,RCM,2023,5'11,154lbs,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,196.4M
5,27.0,Belgium,91.0,91,Chelsea,93M,340K,2142,Right,4.0,4.0,4.0,LF,2020,5'8,163lbs,83+3,83+3,83+3,89+3,88+3,88+3,88+3,89+3,89+3,89+3,89+3,89+3,82+3,82+3,82+3,89+3,66+3,63+3,63+3,63+3,66+3,60+3,49+3,49+3,49+3,60+3,81.0,84.0,61.0,89.0,80.0,95.0,83.0,79.0,83.0,94.0,94.0,88.0,95.0,90.0,94.0,82.0,56.0,83.0,66.0,80.0,54.0,41.0,87.0,89.0,86.0,91.0,34.0,27.0,22.0,172.1M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,19.0,England,47.0,65,Crewe Alexandra,60K,1K,1307,Right,1.0,2.0,2.0,CM,2019,5'9,134lbs,42+2,42+2,42+2,44+2,44+2,44+2,44+2,44+2,45+2,45+2,45+2,44+2,45+2,45+2,45+2,44+2,44+2,45+2,45+2,45+2,44+2,45+2,45+2,45+2,45+2,45+2,34.0,38.0,40.0,49.0,25.0,42.0,30.0,34.0,45.0,43.0,54.0,57.0,60.0,49.0,76.0,43.0,55.0,40.0,47.0,38.0,46.0,46.0,39.0,52.0,43.0,45.0,40.0,48.0,47.0,143K
18203,19.0,Sweden,47.0,63,Trelleborgs FF,60K,1K,1098,Right,1.0,2.0,2.0,ST,2020,6'3,170lbs,45+2,45+2,45+2,39+2,42+2,42+2,42+2,39+2,40+2,40+2,40+2,38+2,35+2,35+2,35+2,38+2,30+2,31+2,31+2,31+2,30+2,29+2,32+2,32+2,32+2,29+2,23.0,52.0,52.0,43.0,36.0,39.0,32.0,20.0,25.0,40.0,41.0,39.0,38.0,40.0,52.0,41.0,47.0,43.0,67.0,42.0,47.0,16.0,46.0,33.0,43.0,42.0,22.0,15.0,19.0,113K
18204,16.0,England,47.0,67,Cambridge United,60K,1K,1189,Right,1.0,3.0,2.0,ST,2021,5'8,148lbs,45+2,45+2,45+2,45+2,46+2,46+2,46+2,45+2,44+2,44+2,44+2,44+2,38+2,38+2,38+2,44+2,34+2,30+2,30+2,30+2,34+2,33+2,28+2,28+2,28+2,33+2,25.0,40.0,46.0,38.0,38.0,45.0,38.0,27.0,28.0,44.0,70.0,69.0,50.0,47.0,58.0,45.0,60.0,55.0,32.0,45.0,32.0,15.0,48.0,43.0,55.0,41.0,32.0,13.0,11.0,165K
18205,17.0,England,47.0,66,Tranmere Rovers,60K,1K,1228,Right,1.0,3.0,2.0,RW,2019,5'10,154lbs,47+2,47+2,47+2,47+2,46+2,46+2,46+2,47+2,45+2

In [ ]:
columns_with_nulls_cleaned()

Series([], dtype: int64)


## Conversión de datos numéricos

Una vez el data set ya no presenta datos con atributos nulos, debemos continuar con la conversión del tipo de los datos que se van a analizar a futuro.

### Conversión de valores monetarios

Como se puede ver, hay columnas que hacen referencia a valores monetarios. Es crucial convertir estas columnas que contienen valores en formato de texto a números, especialmente porque van a ser usados para nuestro modelo predictivo.

Las columnas en cuestión son: Value, Wage y Release Clause.

In [ ]:
def convert_currency(val):
  # Si val ya es numérico (int o float), simplemente devolverlo.
  if isinstance(val, (int, float)):
    return val

  # Asumir que si val no es numérico, debe ser una cadena.
  # Si no es una cadena o tiene menos de 2 caracteres, devolver 0.
  if not isinstance(val, str) or len(val) < 2:
    return 0

  val = val[1:]
  val_r = 0

  if 'M' in val:
    val_r = float(val.replace('M', ''))
    val_r = val_r * 1e6
  elif 'K' in val:
    val_r = float(val.replace('K', ''))
    val_r = val_r * 1e3
  else:
    # Si el valor no contiene ni 'M' ni 'K', intentamos convertirlo directamente a float.
    try:
      return float(val)
    except ValueError:
      return 0

  return val_r

In [ ]:
for col in ['Value', 'Wage', 'Release Clause']:
  cleaned_df[col] = cleaned_df[col].apply(convert_currency)

<ipython-input-174-b9c89ad1e3dc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df[col] = cleaned_df[col].apply(convert_currency)


In [ ]:
cleaned_df[['Value', 'Wage', 'Release Clause']]

,Value,Wage,Release Clause
0,110500000.0,565000.0,226500000.0
1,77000000.0,405000.0,127100000.0
2,118500000.0,290000.0,228100000.0
4,102000000.0,355000.0,196400000.0
5,93000000.0,340000.0,172100000.0
...,...,...,...
18202,60000.0,1000.0,143000.0
18203,60000.0,1000.0,113000.0
18204,60000.0,1000.0,165000.0
18205,60000.0,1000.0,143000.0


### Conversión de calidad en diferentes posiciones

Las columnas 'LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM',
       'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB',
       'LCB', 'CB', 'RCB', 'RB'; hacen referencia a la valoración de un jugador para jugar en esas posiciones, las cuales, en su mayoría, no son su posición natural.

Estas columnas presentan valores con formato X+Y por lo que remplazaremos ese valor con el resultado de dicha suma

In [ ]:
cleaned_df.columns

Index(['Age', 'Nationality', 'Overall', 'Potential', 'Club', 'Value', 'Wage',
       'Special', 'Preferred Foot', 'International Reputation', 'Weak Foot',
       'Skill Moves', 'Position', 'Contract Valid Until', 'Height', 'Weight',
       'LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM',
       'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB',
       'LCB', 'CB', 'RCB', 'RB', 'Crossing', 'Finishing', 'HeadingAccuracy',
       'ShortPassing', 'Volleys', 'Dribbling', 'Curve', 'FKAccuracy',
       'LongPassing', 'BallControl', 'Acceleration', 'SprintSpeed', 'Agility',
       'Reactions', 'Balance', 'ShotPower', 'Jumping', 'Stamina', 'Strength',
       'LongShots', 'Aggression', 'Interceptions', 'Positioning', 'Vision',
       'Penalties', 'Composure', 'Marking', 'StandingTackle', 'SlidingTackle',
       'Release Clause'],
      dtype='object')

In [ ]:
def sum_position(pos):

  num = pos.split("+")

  if len(num) < 2:
    try:
      return num[0]
    except:
      return 0

  res = float(num[0]) + float(num[1])

  return res


In [ ]:
for col in ['LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM',
       'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB',
       'LCB', 'CB', 'RCB', 'RB']:
  cleaned_df[col] = cleaned_df[col].apply(sum_position)

<ipython-input-178-228e858fbedf>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df[col] = cleaned_df[col].apply(sum_position)


In [ ]:
cleaned_df

,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Position,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,Release Clause
0,31.0,Argentina,94.0,94,FC Barcelona,110500000.0,565000.0,2202,Left,5.0,4.0,4.0,RF,2021,5'7,159lbs,90.0,90.0,90.0,94.0,95.0,95.0,95.0,94.0,95.0,95.0,95.0,93.0,86.0,86.0,86.0,93.0,66.0,63.0,63.0,63.0,66.0,61.0,49.0,49.0,49.0,61.0,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,226500000.0
1,33.0,Portugal,94.0,94,Juventus,77000000.0,405000.0,2228,Right,5.0,4.0,5.0,ST,2022,6'2,183lbs,94.0,94.0,94.0,92.0,93.0,93.0,93.0,92.0,91.0,91.0,91.0,91.0,84.0,84.0,84.0,91.0,68.0,64.0,64.0,64.0,68.0,64.0,56.0,56.0,56.0,64.0,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,127100000.0
2,26.0,Brazil,92.0,93,Paris Saint-Germain,118500000.0,290000.0,2143,Right,5.0,5.0,5.0,LW,2022,5'9,150lbs,87.0,87.0,87.0,92.0,92.0,92.0,92.0,92.0,92.0,92.0,92.0,91.0,84.0,84.0,84.0,91.0,68.0,63.0,63.0,63.0,68.0,63.0,50.0,50.0,50.0,63.0,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,228100000.0
4,27.0,Belgium,91.0,92,Manchester City,102000000.0,355000.0,2281,Right,4.0,5.0,4.0,RCM,2023,5'11,154lbs,85.0,85.0,85.0,90.0,90.0,90.0,90.0,90.0,91.0,91.0,91.0,91.0,90.0,90.0,90.0,91.0,80.0,80.0,80.0,80.0,80.0,76.0,69.0,69.0,69.0,76.0,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,196400000.0
5,27.0,Belgium,91.0,91,Chelsea,93000000.0,340000.0,2142,Right,4.0,4.0,4.0,LF,2020,5'8,163lbs,86.0,86.0,86.0,92.0,91.0,91.0,91.0,92.0,92.0,92.0,92.0,92.0,85.0,85.0,85.0,92.0,69.0,66.0,66.0,66.0,69.0,63.0,52.0,52.0,52.0,63.0,81.0,84.0,61.0,89.0,80.0,95.0,83.0,79.0,83.0,94.0,94.0,88.0,95.0,90.0,94.0,82.0,56.0,83.0,66.0,80.0,54.0,41.0,87.0,89.0,86.0,91.0,34.0,27.0,22.0,172100000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,19.0,England,47.0,65,Crewe Alexandra,60000.0,1000.0,1307,Right,1.0,2.0,2.0,CM,2019,5'9,134lbs,44.0,44.0,44.0,46.0,46.0,46.0,46.0,46.0,47.0,47.0,47.0,46.0,47.0,47.0,47.0,46.0,46.0,47.0,47.0,47.0,46.0,47.0,47.0,47.0,47.0,47.0,34.0,38.0,40.0,49.0,25.0,42.0,30.0,34.0,45.0,43.0,54.0,57.0,60.0,49.0,76.0,43.0,55.0,40.0,47.0,38.0,46.0,46.0,39.0,52.0,43.0,45.0,40.0,48.0,47.0,143000.0
18203,19.0,Sweden,47.0,63,Trelleborgs FF,60000.0,1000.0,1098,Right,1.0,2.0,2.0,ST,2020,6'3,170lbs,47.0,47.0,47.0,41.0,44.0,44.0,44.0,41.0,42.0,42.0,42.0,40.0,37.0,37.0,37.0,40.0,32.0,33.0,33.0,33.0,32.0,31.0,34.0,34.0,34.0,31.0,23.0,52.0,52.0,43.0,36.0,39.0,32.0,20.0,25.0,40.0,41.0,39.0,38.0,40.0,52.0,41.0,47.0,43.0,67.0,42.0,47.0,16.0,46.0,33.0,43.0,42.0,22.0,15.0,19.0,113000.0
18204,16.0,England,47.0,67,Cambridge United,60000.0,1000.0,1189,Right,1.0,3.0,2.0,ST,2021,5'8,148lbs,47.0,47.0,47.0,47.0,48.0,48.0,48.0,47.0,46.0,46.0,46.0,46.0,40.0,40.0,40.0,46.0,36.0,32.0,32.0,32.0,36.0,35.0,30.0,30.0,30.0,35.0,25.0,40.0,46.0,38.0,38.0,45.0,38.0,27.0,28.0,44.0,70.0,69.0,50.0,47.0,58.0,45.0,60.0,55.0,32.0,45.0,32.0,15.0,48.0,43.0,55.0,41.0,32.0,13.0,11.0,165000.0
18205,17.0,England,47.0,66,Tranmere Rovers,60000.0,

### Conversión de Height y Weight

Las columnas de Heigth y Weight tienen la falla de presentar unidades, por lo que lo mejor será mostrar solo su representación numérica.

In [ ]:
#Convertir Height a cm
def height_to_cm(height_str):
    # Dividir la cadena en pies y pulgadas
    feet, inches = map(int, height_str.split("'"))

    # Convertir a centímetros
    total_inches = (feet * 12) + inches
    cm = total_inches * 2.54  # 1 pulgada es aproximadamente 2.54 cm

    return cm

In [ ]:
cleaned_df['Height'] = cleaned_df['Height'].apply(height_to_cm)

<ipython-input-197-8b99ea1b2018>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['Height'] = cleaned_df['Height'].apply(height_to_cm)


In [ ]:
#Convertir Weight a kg
def weight_to_kg(weight_str):
    # Eliminar el sufijo 'lbs' y convertir a float
    pounds = float(weight_str.replace('lbs', ''))

    # Convertir a kilogramos
    kg = pounds * 0.453592  # 1 libra es aproximadamente 0.453592 kilogramos

    return kg

In [ ]:
cleaned_df['Weight'] = cleaned_df['Weight'].apply(weight_to_kg)

<ipython-input-199-de7171b8e871>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['Weight'] = cleaned_df['Weight'].apply(weight_to_kg)


## Conversión de columnas categóricas

Finalmente, el dataset presenta 4 columnas categóricas: Nationality, Club, Preferred Foot y Position

Empecemos por el más sencillo, Preferred Foot.

Esta columna solo presenta dos respuestas: Left y Right, por lo que remplazaremos Left por 0 y Right por 1

In [ ]:
cleaned_df['Preferred Foot'] = cleaned_df['Preferred Foot'].replace({'Right': 1, 'Left': 0})

<ipython-input-180-6d078d88a91f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['Preferred Foot'] = cleaned_df['Preferred Foot'].replace({'Right': 1, 'Left': 0})


In [ ]:
cleaned_df

,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Position,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,Release Clause
0,31.0,Argentina,94.0,94,FC Barcelona,110500000.0,565000.0,2202,0,5.0,4.0,4.0,RF,2021,5'7,159lbs,90.0,90.0,90.0,94.0,95.0,95.0,95.0,94.0,95.0,95.0,95.0,93.0,86.0,86.0,86.0,93.0,66.0,63.0,63.0,63.0,66.0,61.0,49.0,49.0,49.0,61.0,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,226500000.0
1,33.0,Portugal,94.0,94,Juventus,77000000.0,405000.0,2228,1,5.0,4.0,5.0,ST,2022,6'2,183lbs,94.0,94.0,94.0,92.0,93.0,93.0,93.0,92.0,91.0,91.0,91.0,91.0,84.0,84.0,84.0,91.0,68.0,64.0,64.0,64.0,68.0,64.0,56.0,56.0,56.0,64.0,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,127100000.0
2,26.0,Brazil,92.0,93,Paris Saint-Germain,118500000.0,290000.0,2143,1,5.0,5.0,5.0,LW,2022,5'9,150lbs,87.0,87.0,87.0,92.0,92.0,92.0,92.0,92.0,92.0,92.0,92.0,91.0,84.0,84.0,84.0,91.0,68.0,63.0,63.0,63.0,68.0,63.0,50.0,50.0,50.0,63.0,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,228100000.0
4,27.0,Belgium,91.0,92,Manchester City,102000000.0,355000.0,2281,1,4.0,5.0,4.0,RCM,2023,5'11,154lbs,85.0,85.0,85.0,90.0,90.0,90.0,90.0,90.0,91.0,91.0,91.0,91.0,90.0,90.0,90.0,91.0,80.0,80.0,80.0,80.0,80.0,76.0,69.0,69.0,69.0,76.0,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,196400000.0
5,27.0,Belgium,91.0,91,Chelsea,93000000.0,340000.0,2142,1,4.0,4.0,4.0,LF,2020,5'8,163lbs,86.0,86.0,86.0,92.0,91.0,91.0,91.0,92.0,92.0,92.0,92.0,92.0,85.0,85.0,85.0,92.0,69.0,66.0,66.0,66.0,69.0,63.0,52.0,52.0,52.0,63.0,81.0,84.0,61.0,89.0,80.0,95.0,83.0,79.0,83.0,94.0,94.0,88.0,95.0,90.0,94.0,82.0,56.0,83.0,66.0,80.0,54.0,41.0,87.0,89.0,86.0,91.0,34.0,27.0,22.0,172100000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,19.0,England,47.0,65,Crewe Alexandra,60000.0,1000.0,1307,1,1.0,2.0,2.0,CM,2019,5'9,134lbs,44.0,44.0,44.0,46.0,46.0,46.0,46.0,46.0,47.0,47.0,47.0,46.0,47.0,47.0,47.0,46.0,46.0,47.0,47.0,47.0,46.0,47.0,47.0,47.0,47.0,47.0,34.0,38.0,40.0,49.0,25.0,42.0,30.0,34.0,45.0,43.0,54.0,57.0,60.0,49.0,76.0,43.0,55.0,40.0,47.0,38.0,46.0,46.0,39.0,52.0,43.0,45.0,40.0,48.0,47.0,143000.0
18203,19.0,Sweden,47.0,63,Trelleborgs FF,60000.0,1000.0,1098,1,1.0,2.0,2.0,ST,2020,6'3,170lbs,47.0,47.0,47.0,41.0,44.0,44.0,44.0,41.0,42.0,42.0,42.0,40.0,37.0,37.0,37.0,40.0,32.0,33.0,33.0,33.0,32.0,31.0,34.0,34.0,34.0,31.0,23.0,52.0,52.0,43.0,36.0,39.0,32.0,20.0,25.0,40.0,41.0,39.0,38.0,40.0,52.0,41.0,47.0,43.0,67.0,42.0,47.0,16.0,46.0,33.0,43.0,42.0,22.0,15.0,19.0,113000.0
18204,16.0,England,47.0,67,Cambridge United,60000.0,1000.0,1189,1,1.0,3.0,2.0,ST,2021,5'8,148lbs,47.0,47.0,47.0,47.0,48.0,48.0,48.0,47.0,46.0,46.0,46.0,46.0,40.0,40.0,40.0,46.0,36.0,32.0,32.0,32.0,36.0,35.0,30.0,30.0,30.0,35.0,25.0,40.0,46.0,38.0,38.0,45.0,38.0,27.0,28.0,44.0,70.0,69.0,50.0,47.0,58.0,45.0,60.0,55.0,32.0,45.0,32.0,15.0,48.0,43.0,55.0,41.0,32.0,13.0,11.0,165000.0
18205,17.0,England,47.0,66,Tranmere Rovers,60000.0,1000.0,1228,1,1.0,3.0,2.0,RW,20

In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)

Ahora para Nationality y Club considero que lo mejor es remplazarlo por la media del valor de sus jugadores

In [ ]:
#Media para Club
club_value_mean = cleaned_df.groupby('Club')['Value'].mean()

KeyError: ignored

In [ ]:
cleaned_df['Mean Club Value'] = cleaned_df['Club'].map(club_value_mean)

<ipython-input-189-800159f54fea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['Mean Club Value'] = cleaned_df['Club'].map(club_value_mean)


In [ ]:
#Eliminamos la columna Club que es categórica
cleaned_df.drop('Club', axis=1, inplace=True)

<ipython-input-190-beaf042dcd98>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df.drop('Club', axis=1, inplace=True)


In [ ]:
cleaned_df

,Age,Nationality,Overall,Potential,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Position,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,Release Clause,Mean Club Value
0,31.00,Argentina,94.00,94,110500000.00,565000.00,2202,0,5.00,4.00,4.00,RF,2021,5'7,159lbs,90.00,90.00,90.00,94.00,95.00,95.00,95.00,94.00,95.00,95.00,95.00,93.00,86.00,86.00,86.00,93.00,66.00,63.00,63.00,63.00,66.00,61.00,49.00,49.00,49.00,61.00,84.00,95.00,70.00,90.00,86.00,97.00,93.00,94.00,87.00,96.00,91.00,86.00,91.00,95.00,95.00,85.00,68.00,72.00,59.00,94.00,48.00,22.00,94.00,94.00,75.00,96.00,33.00,28.00,26.00,226500000.00,26745689.66
1,33.00,Portugal,94.00,94,77000000.00,405000.00,2228,1,5.00,4.00,5.00,ST,2022,6'2,183lbs,94.00,94.00,94.00,92.00,93.00,93.00,93.00,92.00,91.00,91.00,91.00,91.00,84.00,84.00,84.00,91.00,68.00,64.00,64.00,64.00,68.00,64.00,56.00,56.00,56.00,64.00,84.00,94.00,89.00,81.00,87.00,88.00,81.00,76.00,77.00,94.00,89.00,91.00,87.00,96.00,70.00,95.00,95.00,88.00,79.00,93.00,63.00,29.00,95.00,82.00,85.00,95.00,28.00,31.00,23.00,127100000.00,29126136.36
2,26.00,Brazil,92.00,93,118500000.00,290000.00,2143,1,5.00,5.00,5.00,LW,2022,5'9,150lbs,87.00,87.00,87.00,92.00,92.00,92.00,92.00,92.00,92.00,92.00,92.00,91.00,84.00,84.00,84.00,91.00,68.00,63.00,63.00,63.00,68.00,63.00,50.00,50.00,50.00,63.00,79.00,87.00,62.00,84.00,84.00,96.00,88.00,87.00,78.00,95.00,94.00,90.00,96.00,94.00,84.00,80.00,61.00,81.00,49.00,82.00,56.00,36.00,89.00,87.00,81.00,94.00,27.00,24.00,33.00,228100000.00,22380555.56
4,27.00,Belgium,91.00,92,102000000.00,355000.00,2281,1,4.00,5.00,4.00,RCM,2023,5'11,154lbs,85.00,85.00,85.00,90.00,90.00,90.00,90.00,90.00,91.00,91.00,91.00,91.00,90.00,90.00,90.00,91.00,80.00,80.00,80.00,80.00,80.00,76.00,69.00,69.00,69.00,76.00,93.00,82.00,55.00,92.00,82.00,86.00,85.00,83.00,91.00,91.00,78.00,76.00,79.00,91.00,77.00,91.00,63.00,90.00,75.00,91.00,76.00,61.00,87.00,94.00,79.00,88.00,68.00,58.00,51.00,196400000.00,25589137.93
5,27.00,Belgium,91.00,91,93000000.00,340000.00,2142,1,4.00,4.00,4.00,LF,2020,5'8,163lbs,86.00,86.00,86.00,92.00,91.00,91.00,91.00,92.00,92.00,92.00,92.00,92.00,85.00,85.00,85.00,92.00,69.00,66.00,66.00,66.00,69.00,63.00,52.00,52.00,52.00,63.00,81.00,84.00,61.00,89.00,80.00,95.00,83.00,79.00,83.00,94.00,94.00,88.00,95.00,90.00,94.00,82.00,56.00,83.00,66.00,80.00,54.00,41.00,87.00,89.00,86.00,91.00,34.00,27.00,22.00,172100000.00,19355535.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,19.00,England,47.00,65,60000.00,1000.00,1307,1,1.00,2.00,2.00,CM,2019,5'9,134lbs,44.00,44.00,44.00,46.00,46.00,46.00,46.00,46.00,47.00,47.00,47.00,46.00,47.00,47.00,47.00,46.00,46.00,47.00,47.00,47.00,46.00,47.00,47.00,47.00,47.00,47.00,34.00,38.00,40.00,49.00,25.00,42.00,30.00,34.00,45.00,43.00,54.00,57.00,60.00,49.00,76.00,43.00,55.00,40.00,47.00,38.00,46.00,46.00,39.00,52.00,43.00,45.00,40.00,48.00,47.00,143000.00,190208.33
18203,19.00,Sweden,47.00,63,60000.00,1000.00,1098,1,1.00,2.00,2.00,ST,2020,6'3,170lbs,47.00,47.00,47.00,41.00,44.00,44.00,44.00,41.00,42.00,42.00,42.00,40.00,37.00,37.00,37.00,40.00,32.00,33.00,33.00,33.00,32.00,31.00,34.00,34.00,34.00,31.00,23.00,52.00,52.00,43.00,36.00,39.00,32.00,20.00,25.00,40.00,41.00,39.00,38.00,40.00,52.00,41.00,47.00,43.00,67.00,42.00,47.00,16.00,46.00,33.00,43.00,42.00,22.00,15.00,19.00,113000.00,267708.33
18204,16.00,England,47.00,67

In [ ]:
#Media para Nationality
nationality_value_mean = cleaned_df.groupby('Nationality')['Value'].mean()

cleaned_df['Mean Nationality Value'] = cleaned_df['Nationality'].map(nationality_value_mean)

#Eliminamos la columna Nationality que es categórica
cleaned_df.drop('Nationality', axis=1, inplace=True)

<ipython-input-193-69d7f4d2b4c3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['Mean Nationality Value'] = cleaned_df['Nationality'].map(nationality_value_mean)
<ipython-input-193-69d7f4d2b4c3>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df.drop('Nationality', axis=1, inplace=True)


In [ ]:
cleaned_df

,Age,Overall,Potential,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Position,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,Release Clause,Mean Club Value,Mean Nationality Value
0,31.00,94.00,94,110500000.00,565000.00,2202,0,5.00,4.00,4.00,RF,2021,5'7,159lbs,90.00,90.00,90.00,94.00,95.00,95.00,95.00,94.00,95.00,95.00,95.00,93.00,86.00,86.00,86.00,93.00,66.00,63.00,63.00,63.00,66.00,61.00,49.00,49.00,49.00,61.00,84.00,95.00,70.00,90.00,86.00,97.00,93.00,94.00,87.00,96.00,91.00,86.00,91.00,95.00,95.00,85.00,68.00,72.00,59.00,94.00,48.00,22.00,94.00,94.00,75.00,96.00,33.00,28.00,26.00,226500000.00,26745689.66,3282685.56
1,33.00,94.00,94,77000000.00,405000.00,2228,1,5.00,4.00,5.00,ST,2022,6'2,183lbs,94.00,94.00,94.00,92.00,93.00,93.00,93.00,92.00,91.00,91.00,91.00,91.00,84.00,84.00,84.00,91.00,68.00,64.00,64.00,64.00,68.00,64.00,56.00,56.00,56.00,64.00,84.00,94.00,89.00,81.00,87.00,88.00,81.00,76.00,77.00,94.00,89.00,91.00,87.00,96.00,70.00,95.00,95.00,88.00,79.00,93.00,63.00,29.00,95.00,82.00,85.00,95.00,28.00,31.00,23.00,127100000.00,29126136.36,5312205.32
2,26.00,92.00,93,118500000.00,290000.00,2143,1,5.00,5.00,5.00,LW,2022,5'9,150lbs,87.00,87.00,87.00,92.00,92.00,92.00,92.00,92.00,92.00,92.00,92.00,91.00,84.00,84.00,84.00,91.00,68.00,63.00,63.00,63.00,68.00,63.00,50.00,50.00,50.00,63.00,79.00,87.00,62.00,84.00,84.00,96.00,88.00,87.00,78.00,95.00,94.00,90.00,96.00,94.00,84.00,80.00,61.00,81.00,49.00,82.00,56.00,36.00,89.00,87.00,81.00,94.00,27.00,24.00,33.00,228100000.00,22380555.56,4660353.68
4,27.00,91.00,92,102000000.00,355000.00,2281,1,4.00,5.00,4.00,RCM,2023,5'11,154lbs,85.00,85.00,85.00,90.00,90.00,90.00,90.00,90.00,91.00,91.00,91.00,91.00,90.00,90.00,90.00,91.00,80.00,80.00,80.00,80.00,80.00,76.00,69.00,69.00,69.00,76.00,93.00,82.00,55.00,92.00,82.00,86.00,85.00,83.00,91.00,91.00,78.00,76.00,79.00,91.00,77.00,91.00,63.00,90.00,75.00,91.00,76.00,61.00,87.00,94.00,79.00,88.00,68.00,58.00,51.00,196400000.00,25589137.93,5253652.85
5,27.00,91.00,91,93000000.00,340000.00,2142,1,4.00,4.00,4.00,LF,2020,5'8,163lbs,86.00,86.00,86.00,92.00,91.00,91.00,91.00,92.00,92.00,92.00,92.00,92.00,85.00,85.00,85.00,92.00,69.00,66.00,66.00,66.00,69.00,63.00,52.00,52.00,52.00,63.00,81.00,84.00,61.00,89.00,80.00,95.00,83.00,79.00,83.00,94.00,94.00,88.00,95.00,90.00,94.00,82.00,56.00,83.00,66.00,80.00,54.00,41.00,87.00,89.00,86.00,91.00,34.00,27.00,22.00,172100000.00,19355535.71,5253652.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,19.00,47.00,65,60000.00,1000.00,1307,1,1.00,2.00,2.00,CM,2019,5'9,134lbs,44.00,44.00,44.00,46.00,46.00,46.00,46.00,46.00,47.00,47.00,47.00,46.00,47.00,47.00,47.00,46.00,46.00,47.00,47.00,47.00,46.00,47.00,47.00,47.00,47.00,47.00,34.00,38.00,40.00,49.00,25.00,42.00,30.00,34.00,45.00,43.00,54.00,57.00,60.00,49.00,76.00,43.00,55.00,40.00,47.00,38.00,46.00,46.00,39.00,52.00,43.00,45.00,40.00,48.00,47.00,143000.00,190208.33,1621613.64
18203,19.00,47.00,63,60000.00,1000.00,1098,1,1.00,2.00,2.00,ST,2020,6'3,170lbs,47.00,47.00,47.00,41.00,44.00,44.00,44.00,41.00,42.00,42.00,42.00,40.00,37.00,37.00,37.00,40.00,32.00,33.00,33.00,33.00,32.00,31.00,34.00,34.00,34.00,31.00,23.00,52.00,52.00,43.00,36.00,39.00,32.00,20.00,25.00,40.00,41.00,39.00,38.00,40.00,52.00,41.00,47.00,43.00,67.00,42.00,47.00,16.00,46.00,33.00,43.00,42.00,22.00,15.00,19.00,113000.00,267708.33,1174365.

Finalmente, para Position decidimos aplicarle una codificación One-Hot

In [ ]:
cleaned_df = pd.get_dummies(cleaned_df, columns=["Position"])
cleaned_df

,Age,Overall,Potential,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,Release Clause,Mean Club Value,Mean Nationality Value,Position_CAM,Position_CB,Position_CDM,Position_CF,Position_CM,Position_LAM,Position_LB,Position_LCB,Position_LCM,Position_LDM,Position_LF,Position_LM,Position_LS,Position_LW,Position_LWB,Position_RAM,Position_RB,Position_RCB,Position_RCM,Position_RDM,Position_RF,Position_RM,Position_RS,Position_RW,Position_RWB,Position_ST
0,31.00,94.00,94,110500000.00,565000.00,2202,0,5.00,4.00,4.00,2021,170.18,72.12,90.00,90.00,90.00,94.00,95.00,95.00,95.00,94.00,95.00,95.00,95.00,93.00,86.00,86.00,86.00,93.00,66.00,63.00,63.00,63.00,66.00,61.00,49.00,49.00,49.00,61.00,84.00,95.00,70.00,90.00,86.00,97.00,93.00,94.00,87.00,96.00,91.00,86.00,91.00,95.00,95.00,85.00,68.00,72.00,59.00,94.00,48.00,22.00,94.00,94.00,75.00,96.00,33.00,28.00,26.00,226500000.00,26745689.66,3282685.56,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,33.00,94.00,94,77000000.00,405000.00,2228,1,5.00,4.00,5.00,2022,187.96,83.01,94.00,94.00,94.00,92.00,93.00,93.00,93.00,92.00,91.00,91.00,91.00,91.00,84.00,84.00,84.00,91.00,68.00,64.00,64.00,64.00,68.00,64.00,56.00,56.00,56.00,64.00,84.00,94.00,89.00,81.00,87.00,88.00,81.00,76.00,77.00,94.00,89.00,91.00,87.00,96.00,70.00,95.00,95.00,88.00,79.00,93.00,63.00,29.00,95.00,82.00,85.00,95.00,28.00,31.00,23.00,127100000.00,29126136.36,5312205.32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,26.00,92.00,93,118500000.00,290000.00,2143,1,5.00,5.00,5.00,2022,175.26,68.04,87.00,87.00,87.00,92.00,92.00,92.00,92.00,92.00,92.00,92.00,92.00,91.00,84.00,84.00,84.00,91.00,68.00,63.00,63.00,63.00,68.00,63.00,50.00,50.00,50.00,63.00,79.00,87.00,62.00,84.00,84.00,96.00,88.00,87.00,78.00,95.00,94.00,90.00,96.00,94.00,84.00,80.00,61.00,81.00,49.00,82.00,56.00,36.00,89.00,87.00,81.00,94.00,27.00,24.00,33.00,228100000.00,22380555.56,4660353.68,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,27.00,91.00,92,102000000.00,355000.00,2281,1,4.00,5.00,4.00,2023,180.34,69.85,85.00,85.00,85.00,90.00,90.00,90.00,90.00,90.00,91.00,91.00,91.00,91.00,90.00,90.00,90.00,91.00,80.00,80.00,80.00,80.00,80.00,76.00,69.00,69.00,69.00,76.00,93.00,82.00,55.00,92.00,82.00,86.00,85.00,83.00,91.00,91.00,78.00,76.00,79.00,91.00,77.00,91.00,63.00,90.00,75.00,91.00,76.00,61.00,87.00,94.00,79.00,88.00,68.00,58.00,51.00,196400000.00,25589137.93,5253652.85,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
5,27.00,91.00,91,93000000.00,340000.00,2142,1,4.00,4.00,4.00,2020,172.72,73.94,86.00,86.00,86.00,92.00,91.00,91.00,91.00,92.00,92.00,92.00,92.00,92.00,85.00,85.00,85.00,92.00,69.00,66.00,66.00,66.00,69.00,63.00,52.00,52.00,52.00,63.00,81.00,84.00,61.00,89.00,80.00,95.00,83.00,79.00,83.00,94.00,94.00,88.00,95.00,90.00,94.00,82.00,56.00,83.00,66.00,80.00,54.00,41.00,87.00,89.00,86.00,91.00,34.00,27.00,22.00,172100000.00,19355535.71,5253652.85,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,19.00,47.00,65,60000.00,1000.00,1307,1,1.00,2.00,2.00,2019,175.26,60.78,44.00,44.00,44.00,46.00,46.00,46.00,46.00,46.00,47.00,47.00,47.00,46.00,47.00,47.00,47.00,46.00,46.00,47.00,47.00,47.00,46.00,47.00

# Guardar el dataset procesado

El propósito de esto notebook era entender el dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

cleaned_df.to_csv("/content/drive/My Drive/fifa-19-cleaned-dataset.csv", index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
